<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Entrenamiento de predictor</h4>
</center>

In [1]:
# Importación librerias

# !pip install awswrangler
# !pip install boto3
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

In [2]:
# Definición fechas

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

timestamp_format="yyyy-MM-dd"
dataset_frequency='M'

# Versión modelo
data_version = '2'
grouped_cols = ["product_id", "cluster_id"]
grouped_cols_name = "product_cluster"

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


In [3]:
# Conexiones AWS

account_id=boto3.client('sts').get_caller_identity().get('Account')
session = boto3.Session() 
forecast = session.client(service_name='forecast')
region = forecast.meta.region_name
forecastquery = session.client(service_name='forecastquery')
role = f'arn:aws:iam::{account_id}:role/ForecastRole'

In [4]:
# Qué valores predecir: media o percentiles
quantiles=['mean']
# Horizonte de predicción
forecast_length = 2
dataset_group_arn=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version}_{grouped_cols_name}'

In [5]:
# Verificar comunicación con Amazon Forecast
assert forecast.list_predictors()

In [6]:
predictor_name_automl = f"modelo_{data_version}_predictor"

El siguiente código es para crear el predictor. Obtenemos error porque fue creado desde la UI para este primer modelo.

In [11]:
response = forecast.create_auto_predictor(
    PredictorName=predictor_name_automl,
    ForecastHorizon=forecast_length,
    ForecastTypes=quantiles,
    ForecastDimensions=["cluster_id"],
    ForecastFrequency=dataset_frequency,
    DataConfig={
        'DatasetGroupArn': dataset_group_arn,
        'AttributeConfigs': [
		{
		"AttributeName": "target_value",
		"Transformations": {
			"aggregation": "sum",
			"backfill": "zero",
			"frontfill": "none",
			"middlefill": "zero"
		}
		}
	]}, 
    TimeAlignmentBoundary={
        'DayOfMonth': 1
    },
    OptimizationMetric='MAPE',
    ExplainPredictor=False
)

In [12]:
automl_predictor_arn_wape = response['PredictorArn']

In [13]:
print(f"Waiting for Predictor with ARN {automl_predictor_arn_wape} to become ACTIVE. \nCurrent Status:")
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape)['Status']

Waiting for Predictor with ARN arn:aws:forecast:us-east-1:637423651905:predictor/modelo_2_predictor_01HZQRGFBR9PMEB0RNENW4X9BQ to become ACTIVE. 
Current Status:


'CREATE_PENDING'